In [1]:
# Import required libraries
import cv2
import numpy as np
from keras.preprocessing import image as img
from keras.models import model_from_yaml
import pickle

Using TensorFlow backend.


In [2]:
def opencv_init():
    '''
        Purpose: Creates the classifier and video capture object
        Arguements: N/A
        Return: classifier and video capture object
    '''
    face_detector_object = cv2.CascadeClassifier('/Users/Iris/DeveloperTools/anaconda3/envs/expression-recognition/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')
    video_capture_object = cv2.VideoCapture(0)
    return (face_detector_object, video_capture_object)

In [3]:
def opencv_cleanup(cap):
    '''
        Purpose: Release resource held by opencv
        Arguements: OpenCV video object
        Return: N/A
    '''
    cap.release()
    cv2.destroyAllWindows()

In [4]:
def load_model():
    '''
        Purpose: Load the saved model
        Arguements: N/A
        Return: Loaded model
    '''
    '''with open("model_cnn.yaml", "r") as yaml_file:
        loaded_model_yaml = yaml_file.read()
    model = model_from_yaml(loaded_model_yaml)
    model.load_weights("model_cnn.h5")
    print("Loaded model from disk")
    return (model)'''
    model = pickle.load(open('/Users/Iris/SJSU/Fall_2018/CMPE_257/Project/Group/local/model/knn/model_knn.sav', 'rb'))
    return(model)

In [5]:
def emotion_detection(face_detect, video_capture, model):
    '''
        Purpose: Begin video cature and perform emotion detection of all faces detected
        Arguements: face_detect, video_capture and model references
        Return: N/A
    '''
    emotion_classes = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    while(True):
        ret, image = video_capture.read()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_detect.detectMultiScale(gray, 1.3, 5)

        print("Detected face positions: {}".format(faces))

        for (x,y,w,h) in faces:
            # Create boundary for the recognirzed face
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            # Crop detected faces
            detected_face = image[int(y):int(y+h), int(x):int(x+w)]

            # Transform the detected face to grayscale
            detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)

            # Resize face image to 48x48
            detected_face = cv2.resize(detected_face, (48, 48))

            img_pixels = img.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 255 
            img_pixels = np.vstack(img_pixels)
            print(img_pixels.reshape(48,48))
            # Obtain predictions for the detected face
            predictions = model.predict(img_pixels.reshape(48,48))

            # Find index of the maximum value 
            # Array - 0: angry, 1:disgust, 2:fear, 3:happy, 4:sad, 5:surprise, 6:neutral
            max_index = np.argmax(predictions[0])

            emotion = emotion_classes[max_index]

            # Display the emotion text above rectangle
            cv2.putText(image, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.imshow('img',image)
        
        # Break video capture event
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [6]:
def main():
    # Intialize opencv and obtain face detection and video capture objects
    face_detect, video_capture = opencv_init()
    
    # Load the model 
    model = load_model()
    
    # Start video capture and emotion recognition
    emotion_detection(face_detect, video_capture, model)
    
    # Deallocate resources used by OpenCV
    opencv_cleanup(video_capture)

In [7]:
if __name__ == "__main__":
    main()

Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: ()
Detected face positions: [[512  20 289 289]]
[[0.7058824  0.5568628  0.2784314  ... 0.11372549 0.18431373 0.1882353 ]
 [0.59607846 0.36862746 0.2627451  ... 0.06666667 0.12156863 0.34901962]
 [0.5019608  0.36078432 0.21568628 ... 0.07058824 0.16470589 0.48235294]
 ...
 [0.36862746 0.37254903 0.3647059  ... 0.7254902  0.7411765  0.73333335]
 [0.35686275 0.35686275 0.8627451  ... 0.72156864 0.7372549  0.7294118 ]
 [0.4745098  0.32941177 0.8627451  ... 0.20784314 0.5

ValueError: query data dimension must match training data dimension